In [19]:
# modules
import os
import shapefile # this is installed as "PyShp" library
import numpy as np
import xarray as xr
import netCDF4 as nc4
import geopandas as gpd
from pathlib import Path
from shutil import copyfile
from datetime import datetime
import matplotlib.pyplot as plt
from utils.read_files import make_default_path, read_from_control

In [20]:
# Store the name of the 'active' file in a variable
controlFile = 'control_TuolumneRiver.txt'

In [21]:
# Find the path where the merged forcing is
mergePath = read_from_control(controlFile,'forcing_merged_path')
# Specify the default path if required
if mergePath == 'default':
    mergePath = make_default_path('forcing/2a_merged_data', controlFile)
else: 
    mergePath = Path(mergePath) # ensure Path() object  

In [22]:
# Find file path
geoPath = read_from_control(controlFile,'forcing_geo_path')
geoName = 'gridmet_dem.nc'
# Specify the default path if required
if geoPath == 'default':
    geoPath = make_default_path('forcing/0_geopotential', controlFile)
else: 
    geoPath = Path(geoPath) # ensure Path() object 

In [23]:
# Find the path where the new shapefile needs to go
shapePath = read_from_control(controlFile,'forcing_shape_path')

# Specify the default path if required
if shapePath == 'default':
    shapePath = make_default_path('shapefiles/forcing', controlFile)
else: 
    shapePath = Path(shapePath) # ensure Path() object 
    
# Find name of the new shapefile
shapeName = read_from_control(controlFile,'forcing_shape_name')

In [24]:
# Find the names of the latitude and longitude fields
field_lat = read_from_control(controlFile,'forcing_shape_lat_name')
field_lon = read_from_control(controlFile,'forcing_shape_lon_name')

In [25]:
# --- Read the source file to find the grid spacing
# Find an .nc file in the forcing path
for file in os.listdir(mergePath):
    if file.endswith('.nc'):
        forcing_file = file
        break

In [26]:
# Set the dimension variable names
source_name_lat = "y"
source_name_lon = "x"

In [27]:
# Open the file and get the dimensions  and thus the spatial extent of the domain
with nc4.Dataset(mergePath / forcing_file) as src:
    lat = src.variables[source_name_lat][:]
    lon = src.variables[source_name_lon][:]

In [28]:
# Find the spacing
half_dlat = abs(lat[1] - lat[0])/2
half_dlon = abs(lon[1] - lon[0])/2

In [29]:
# --- Create the new shape
with shapefile.Writer(str(shapePath / shapeName)) as w:
    w.autoBalance = 1 # turn on function that keeps file stable if number of shapes and records don't line up
    w.field("ID",'N') # create (N)umerical attribute fields, integer
    w.field(field_lat,'F',decimal=4) # float with 4 decimals
    w.field(field_lon,'F',decimal=4)
    ID = 0 # start ID counter of empty
    
    for i in range(0,len(lon)):
        for j in range(0,len(lat)):
            ID += 1
            center_lon = lon[i]
            center_lat = lat[j]
            vertices = []
            parts = []
            vertices.append([center_lon-half_dlon, center_lat])
            vertices.append([center_lon-half_dlon, center_lat+half_dlat])
            vertices.append([center_lon          , center_lat+half_dlat])
            vertices.append([center_lon+half_dlon, center_lat+half_dlat])
            vertices.append([center_lon+half_dlon, center_lat])
            vertices.append([center_lon+half_dlon, center_lat-half_dlat])
            vertices.append([center_lon          , center_lat-half_dlat])
            vertices.append([center_lon-half_dlon, center_lat-half_dlat])
            vertices.append([center_lon-half_dlon, center_lat])
            parts.append(vertices)
            w.poly(parts)
            w.record(ID, center_lat, center_lon)
            

In [30]:
# --- Add the geopotential data to the shape
# Open the geopotential data file
geo = xr.open_dataset( geoPath / geoName )

# Open shapefile
shp = gpd.read_file( shapePath / shapeName )

# write crs to shp
shp = shp.set_crs("EPSG:4326")

# Add new column to shapefile
shp = shp.assign(elev_m = -999)  # insert a placeholder value

# For each row in the shapefile, match its ERA5 lat/lon coordinates 
# with those in the 'geo' file and extract the appropriate geopotential
for index, row in shp.iterrows():
       
    # Find elevation
    elev = geo['elevation'].sel(y = row['y'], x=row['x'], method='nearest').values.flatten()
    
    # Add elevation into shapefile
    shp.at[index,'elev_m'] = int(elev[0])
    
# Overwrite the existing shapefile
shp.to_file( shapePath / shapeName )

# close the files
geo.close()
shp = []